## Import

In [ ]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder
from matplotlib.dates import DateFormatter, MonthLocator

import matplotlib.pyplot as plt
import seaborn as sns

# Train, prodcut_info concat

In [ ]:
train_data = pd.read_csv('/content/data/train.csv').drop(columns=['ID'])
train_data = train_data.iloc[:, :6]

product = pd.read_csv('/content/data/product_info.csv')
product = product[['제품', '제품특성']]

In [ ]:
train_product = train_data['제품']
product_only = product['제품']

In [ ]:
# 교집합 구하기
intersection = set(train_product).intersection(set(product_only))

# 교집합의 개수 출력
print(len(intersection))

9158


In [ ]:
# 차집합 구하기
train_diff = set(train_product) - set(product_only)
product_diff = set(product_only) - set(train_product)

# 차집합의 개수 출력
print('train의 갯수:', len(train_diff))
print('product 갯수:', len(product_diff))

train의 갯수: 4738
product 갯수: 3620


In [ ]:
merged = train_data.merge(product[['제품', '제품특성']], on='제품', how='left')

In [ ]:
merged.fillna(0)

,제품,대분류,중분류,소분류,브랜드,쇼핑몰,제품특성
0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,S001-00001,헤어타입:모든 모발용 제품형태:스프레이형 주요제품특징:머릿결개선 주요제품특징:흡수력...
1,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0
2,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0
3,B002-00002-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0
4,B002-00002-00004,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00010,0
...,...,...,...,...,...,...,...
28889,B002-03798-00046,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03798,S001-00001,0
28890,B002-03799-00002,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,S001-00001,0
28891,B002-03799-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,S001-00001,0
28892,B002-03799-00004,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,S001-00001,종류:이유식보관용기 특징:전자레인지사용 재질:실리콘 3개세트 LLG508S3


In [ ]:
# 10,481개만 값이 존재
merged.isnull().sum()

제품      0
대분류     0
중분류     0
소분류     0
브랜드     0
쇼핑몰     0
제품특성    0
dtype: int64

In [ ]:
merged['제품특성'] = merged.groupby(['대분류', '중분류', '소분류'])['제품특성'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else None))

In [ ]:
merged.isnull().sum()

제품         0
대분류        0
중분류        0
소분류        0
브랜드        0
쇼핑몰        0
제품특성    9782
dtype: int64

## **Tokenization**

In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 48.7 MB/s eta 0:00:00


In [ ]:
import gensim
from gensim import corpora
from nltk.tokenize import word_tokenize
import nltk
import re
from konlpy.tag import Okt
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
merged['제품특성'] = merged['제품특성'].apply(lambda x: [word for word in nltk.word_tokenize(str(x)) if word not in ['(', ',', ')', ':', '%', '&']])
merged

,제품,대분류,중분류,소분류,브랜드,쇼핑몰,제품특성
0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,S001-00001,"[헤어타입, 모든, 모발용, 제품형태, 스프레이형, 주요제품특징, 머릿결개선, 주요..."
1,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,"[특징, 방수기능, 사용횟수, 다회용, 재질, PP]"
2,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,"[특징, 방수기능, 사용횟수, 다회용, 재질, PP]"
3,B002-00002-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,"[특징, 방수기능, 사용횟수, 다회용, 재질, PP]"
4,B002-00002-00004,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00010,"[특징, 방수기능, 사용횟수, 다회용, 재질, PP]"
...,...,...,...,...,...,...,...
28889,B002-03798-00046,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03798,S001-00001,"[특징, 방수기능, 사용횟수, 다회용, 재질, PP]"
28890,B002-03799-00002,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,S001-00001,"[용도, 가정용, 살균방식, 자외선, 기능, 건조기능, 기능, 살균기능, 기능, 멸..."
28891,B002-03799-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,S001-00001,"[특징, 방수기능, 사용횟수, 다회용, 재질, PP]"
28892,B002-03799-00004,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,S001-00001,"[종류, 이유식보관용기, 특징, 전자레인지사용, 재질, 실리콘, 3개세트, LLG5..."


In [ ]:
for i in merged['제품특성']:
  print(i)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['칼슘:300mg', '영양소', '원료명', '식약처고시', '비타민D', '영양소', '원료명', '식약처고시', '칼슘', '영양소', '원료명', '식약처고시', '마그네슘', '영양소', '원료명', '식약처고시', '아연', '영양소', '원료명', '식약처고시', '망간', '섭취대상', '성인남녀', '1일', '총', '섭취량:1정', '제품용량:3개월분', '제품타입', '정', '섭취횟수', '하루', '한', '번', '섭취방법', '물과', '함께', '주요', '기능성', '식약처인증', '관절/뼈건강', ':1개', '칼슘:300mg', '영양소', '원료명', '식약처고시', '비타민D', '영양소', '원료명', '식약처고시', '칼슘', '영양소', '원료명', '식약처고시', '마그네슘', '영양소', '원료명', '식약처고시', '아연', '섭취대상', '성인남녀', '1일', '총', '섭취량:2정', '제품용량:1개월분', '제품타입', '정', '섭취횟수', '하루', '한', '번', '섭취방법', '물과', '함께', '주요', '기능성', '식약처인증', '관절/뼈건강', ':1개', '칼슘:300mg', '영양소', '원료명', '식약처고시', '비타민D', '영양소', '원료명', '식약처고시', '칼슘', '영양소', '원료명', '식약처고시', '마그네슘', '영양소', '원료명', '식약처고시', '아연', '섭취대상', '유아/청소년', '1일', '총', '섭취량:2정', '제품용량:3개월분', '제품타입', '츄어블', '섭취횟수', '하루', '두', '번', '섭취방법', '씹어서', '주요', '기능성', '식약처인증', '관절/뼈건강', ':1개', '칼슘:300mg', '영양소', '원료명', '식약처고시', '비타민D', '영양소', '원료명', '식약처고시', '칼슘', '영양소', '원료명', '식약처고시', '마그네슘', '영양소', '원료명', '식약처고시', '아연', '영양소'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## **middle split**

In [ ]:
unique_subcategories = merged['중분류'].unique()  # 소분류 컬럼의 고유한 값들을 가져옵니다.

for subcategory in unique_subcategories:
    subcategory_data = merged[merged['중분류'] == subcategory]  # 소분류에 해당하는 데이터를 필터링합니다.

    # 3. 분리된 데이터 저장
    subcategory_filename = f"/content/data/middle/{subcategory}_data.csv"  # 각 소분류별로 파일명을 지정합니다.
    subcategory_data.to_csv(subcategory_filename, index=False)

## **sub split**

In [ ]:
unique_subcategories = merged['소분류'].unique()  # 소분류 컬럼의 고유한 값들을 가져옵니다.

for subcategory in unique_subcategories:
    subcategory_data = merged[merged['소분류'] == subcategory]  # 소분류에 해당하는 데이터를 필터링합니다.

    # 3. 분리된 데이터 저장
    subcategory_filename = f"/content/data/sub/{subcategory}_data.csv"  # 각 소분류별로 파일명을 지정합니다.
    subcategory_data.to_csv(subcategory_filename, index=False)

In [ ]:
product_info

,제품,제품특성
0,B002-03509-00001,제품유형:일반식품 콜라겐 펩타이드:1000mg 종류:어류 분자량:300Da 섭취대상...
1,B002-02376-00001,700mg x 28정
2,B002-03735-00001,제품타입:정 섭취방법:물과 함께 섭취대상:성인남녀 섭취횟수:하루 두 번 1일 총 섭...
3,B002-03735-00003,제품타입:정 섭취방법:물과 함께 섭취대상:성인남녀 섭취횟수:하루 한 번 1일 총 섭...
4,B002-02769-00001,HCA:900mg 영양소 원료명(식약처고시):비타민B1 영양소 원료명(식약처고시):...
...,...,...
12773,B002-01994-00001,형태:액상형 구성:리필 등급:1종 용도:식기 용도:과일 용도:야채 용도:조리기구 용...
12774,B002-02600-00002,헤어타입:모든 모발용 주요제품특징:머릿결개선 세부제품특징:촉촉함(수분공급) 세부제품...
12775,B002-02372-00095,사용대상:여성용 타입:일자형 흡수량:20ml :1팩 32개
12776,B002-01318-00002,피부타입:모든피부용 향계열:무향 주요제품특징:촉촉함(수분공급) 주요제품특징:풍부한 ...


# **LDA**

In [ ]:
sub_28 = pd.read_csv('/content/data/sub/B002-C003-0028_data.csv')
sub_29 = pd.read_csv('/content/data/sub/B002-C003-0029_data.csv')
sub_41 = pd.read_csv('/content/data/sub/B002-C003-0041_data.csv')
sub_52 = pd.read_csv('/content/data/sub/B002-C003-0052_data.csv')
sub_53 = pd.read_csv('/content/data/sub/B002-C003-0053_data.csv')

In [ ]:
for i in sub_28['제품특성']:
  print(i)

In [ ]:
for i in sub_29['제품특성']:
  print(i)

In [ ]:
for i in sub_41['제품특성']:
  print(i)

In [ ]:
for i in sub_52['제품특성']:
  print(i)

In [ ]:
for i in sub_53['제품특성']:
  print(i)

## **중분류**

In [ ]:
label_start = 1

for i in range(1, 2):
    num_str = f"{i:02}"

    df = pd.read_csv(f'/content/data/middle/B002-C002-00{num_str}_data.csv')

    p_1 = df['제품특성'].apply(lambda x: [word.strip("'") for word in nltk.word_tokenize(str(x)) if word not in ["''",'``',"'",'[', ',', ']', ':', '%', '&']])

    # 불용어 처리
    stop_words = set([
    "유형", "총","물과",":1개","물에","두","급여방법","저자극","성인남녀","x","제품유형","한","부가기능","제품타입사이즈","유발성분",":10팩","제품형태",":1팩","대상","g","성분" ,"구성용량","사용횟수","본품","형태","주요재질","식약처인증","하루","섭취횟수","식약처고시","영양소","원료명","횟수", "사용", "번","방법", "사용방법", "바로", "정", "기능", "함께", "주요","일반", "타입",
    "종류", '중량',"원료명","섭취대상","안전기능","섭취방법",'교체','작동', '균일', '시즌', '증정', "밸런스","패턴", "상태", "가능성", "개인", "개당", "자동", "상관없음",
    "권장", "구분", "복합", "개선", "전용", "1일", ":1개","1개","방문", "부가", "이력", "모든", "수량", "출력", "기타", "필수", "해당", "시스템", "수시로", "제품", "품목",
    "구성","세트", "타입","세부제품특징","주요제품특징","주요제품특징","주요","사용", "적용", "기능","특징", "혜택","조건", "단계", "용도", "과정","타겟",
    "모드", "관리", "기능성","고정방식","편의기능","형태사용횟수주성분","무첨가","보관방법","최소연령",'포장형태','유발성분알레르기','인증', "부가기능","세트구성","크기", "재질","제품타입", "편의", "표시", "중심", "모형", "공용", "개월", "세부", "인기", "상품", "효과", "무게", "디테일", "기본", "편리", "인용","정성",
    "전체", "규격" ])

    p_1 = p_1.apply(lambda x: [word for word in x if word.lower() not in stop_words])

    # 데이터 전처리 후, 딕셔너리와 코퍼스 생성
    dictionary = Dictionary(p_1)
    corpus = [dictionary.doc2bow(text) for text in p_1]

    # 토픽 수 범위 설정
    start_topic_count = 1
    end_topic_count = 10
    step = 1

    # 일관성 점수 및 퍼플렉시티 계산
    coherence_scores = []
    perplexity_scores = []

    for num_topics in range(start_topic_count, end_topic_count + 1, step):
        model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary)

        coherence_model = CoherenceModel(model=model, texts=p_1, dictionary=dictionary, coherence='c_v')
        coherence_score = coherence_model.get_coherence()
        coherence_scores.append((num_topics, coherence_score))

        perplexity_score = model.log_perplexity(corpus)
        perplexity_scores.append((num_topics, perplexity_score))

        # 토픽 별 단어 출력
        print(f"--- {num_topics} Topics ---")
        for topic_id, words in model.print_topics(num_words=5):
            print(f"Topic {topic_id}: {words}")

    print(f'------------------------{i}번 제품------------------------')


    # coherence 점수 상위 3개 중 perplexity가 가장 높은 토픽 선택
    top_coherence_scores = sorted(coherence_scores, key=lambda x: x[1], reverse=True)[:3]
    best_perplexity_num_topics = max(top_coherence_scores, key=lambda x: x[1])[0]

    print(f"Best Topics for Coherence: {[score[0] for score in top_coherence_scores]}")
    print(f"Best Topics for Perplexity: {best_perplexity_num_topics}")

    best_model = LdaModel(corpus, num_topics=best_perplexity_num_topics, id2word=dictionary)

    # 각 문서에 대한 토픽 분포 가져오기
    doc_topic_dists = [best_model.get_document_topics(doc) for doc in corpus]

    # 가장 확률이 높은 토픽 선택하기
    labels = [max(dist, key=lambda x: x[1])[0] + label_start for dist in doc_topic_dists]

    # 데이터프레임에 'label' 열 추가하기
    df['label'] = labels

    label_start += best_perplexity_num_topics

    # Save the labeled data to a CSV file
    output_path = f'/content/data/middle/B002-C002-00{num_str}_data.csv'
    df.to_csv(output_path, index=False)

--- 1 Topics ---
Topic 0: 0.038*"비타민C" + 0.029*"아연" + 0.025*"비타민D" + 0.022*"비타민E" + 0.021*"비타민B1"


--- 2 Topics ---
Topic 0: 0.045*"비타민C" + 0.033*"아연" + 0.027*"비타민D" + 0.025*"비타민B1" + 0.024*"비타민E"
Topic 1: 0.035*"제품용량:1개월분" + 0.029*"음용" + 0.027*"섭취량:1포" + 0.025*"캡슐" + 0.022*"건강기능식품"


--- 3 Topics ---
Topic 0: 0.056*"비타민C" + 0.039*"아연" + 0.031*"비타민D" + 0.030*"비타민B1" + 0.029*"비타민E"
Topic 1: 0.034*"음용" + 0.031*"섭취량:1포" + 0.025*"건강기능식품" + 0.023*"홍삼농축액" + 0.021*"면역력"
Topic 2: 0.044*"단백질성분" + 0.027*"캡슐" + 0.025*"제품용량:1개월분" + 0.020*"섭취량:1캡슐" + 0.019*"섞어서"


--- 4 Topics ---
Topic 0: 0.034*"음용" + 0.027*"비타민C" + 0.024*"분말" + 0.023*"제품용량:1개월분" + 0.022*"캡슐"
Topic 1: 0.080*"단백질성분" + 0.039*"섞어서" + 0.037*"맛" + 0.037*"단백질종류" + 0.023*"동물성+식물성"
Topic 2: 0.053*"비타민C" + 0.039*"아연" + 0.033*"비타민D" + 0.032*"비타민E" + 0.030*"비타민B2"
Topic 3: 0.057*"섭취량:1포" + 0.046*"음용" + 0.029*"제품용량:1개월분" + 0.027*"장건강" + 0.026*"CFU"


--- 5 Topics ---
Topic 0: 0.071*"단백질성분" + 0.038*"섞어서" + 0.030*"단백질종류" + 0.030*"맛" + 0.028*"섭취량:1포"
Topic 1: 0.040*"음용" + 0.035*"제품용량:1개월분" + 0.032*"섭취량:1포" + 0.028*"건강기능식품" + 0.026*"비타민C"
Topic 2: 0.058*"비타민C" + 0.038*"아연" + 0.032*"비타민D" + 0.030*"비타민E" + 0.030*"비타민B6"
Topic 3: 0.039*"캡슐" + 0.036*"제품용량:2개월분" + 0.032*"CFU" + 0.029*"섭취량:1캡슐" + 0.029*"장건강"
Topic 4: 0.034*"캡슐" + 0.028*"제품용량:1개월분" + 0.027*"아연" + 0.026*"비타민B2" + 0.024*"섭취량:1캡슐"


--- 6 Topics ---
Topic 0: 0.035*"아연" + 0.032*"비타민C" + 0.024*"비타민B2" + 0.023*"비타민B6" + 0.023*"비타민D"
Topic 1: 0.039*"건강기능식품" + 0.036*"제품용량:1개월분" + 0.033*"음용" + 0.032*"없음" + 0.026*"홍삼농축액"
Topic 2: 0.042*"비타민C" + 0.037*"아연" + 0.036*"캡슐" + 0.029*"섭취량:1캡슐" + 0.028*"제품용량:2개월분"
Topic 3: 0.055*"비타민C" + 0.035*"아연" + 0.034*"비타민D" + 0.031*"비타민E" + 0.031*"비타민B6"
Topic 4: 0.056*"캡슐" + 0.040*"제품용량:1개월분" + 0.039*"비타민E" + 0.035*"비타민D" + 0.034*"섭취량:1캡슐"
Topic 5: 0.057*"단백질성분" + 0.032*"섭취량:1포" + 0.031*"섞어서" + 0.030*"음용" + 0.024*"맛"


--- 7 Topics ---
Topic 0: 0.069*"음용" + 0.057*"고형분" + 0.054*"건강기능식품" + 0.051*"홍삼농축액" + 0.047*"면역력"
Topic 1: 0.040*"아연" + 0.038*"비타민D" + 0.032*"비타민B6" + 0.031*"단백질성분" + 0.026*"비타민B1"
Topic 2: 0.055*"비타민C" + 0.041*"아연" + 0.034*"비타민D" + 0.032*"비타민E" + 0.031*"비타민B2"
Topic 3: 0.097*"단백질성분" + 0.055*"섞어서" + 0.053*"맛" + 0.052*"단백질종류" + 0.030*"동물성+식물성"
Topic 4: 0.070*"섭취량:1포" + 0.065*"음용" + 0.049*"제품용량:1개월분" + 0.046*"분말" + 0.046*"CFU"
Topic 5: 0.039*"캡슐" + 0.036*"제품용량:1개월분" + 0.027*"비타민E" + 0.026*"일반식품" + 0.024*"비타민D"
Topic 6: 0.044*"비타민C" + 0.026*"캡슐" + 0.023*"제품용량:2개월분" + 0.021*"아연" + 0.020*"섭취량:1캡슐"


--- 8 Topics ---
Topic 0: 0.051*"비타민C" + 0.035*"비타민E" + 0.033*"비타민D" + 0.033*"아연" + 0.032*"비타민A"
Topic 1: 0.056*"비타민C" + 0.048*"아연" + 0.033*"비타민B1" + 0.032*"비타민B2" + 0.029*"비타민B6"
Topic 2: 0.060*"섭취량:1포" + 0.056*"음용" + 0.042*"건강기능식품" + 0.036*"일반식품" + 0.035*"제품용량:1개월분"
Topic 3: 0.041*"아연" + 0.035*"비타민C" + 0.033*"비타민B6" + 0.033*"비타민B2" + 0.032*"영양보충"
Topic 4: 0.032*"음용" + 0.031*"비타민C" + 0.030*"아연" + 0.027*"섭취량:1포" + 0.025*"비타민D"
Topic 5: 0.075*"CFU" + 0.075*"장건강" + 0.050*"제품용량:2개월분" + 0.047*"섭취량:1캡슐" + 0.046*"캡슐"
Topic 6: 0.095*"단백질성분" + 0.048*"섞어서" + 0.042*"맛" + 0.042*"단백질종류" + 0.027*"농축유청단백질"
Topic 7: 0.038*"분말" + 0.035*"비타민C" + 0.031*"제품용량" + 0.026*"일반식품" + 0.021*"섭취량"


--- 9 Topics ---
Topic 0: 0.073*"음용" + 0.071*"홍삼농축액" + 0.060*"면역력" + 0.060*"건강기능식품" + 0.056*"고형분"
Topic 1: 0.094*"단백질성분" + 0.045*"단백질종류" + 0.044*"섞어서" + 0.043*"맛" + 0.028*"동물성+식물성"
Topic 2: 0.025*"단백질성분" + 0.024*"비타민D" + 0.023*"비타민C" + 0.023*"비타민B1" + 0.022*"비타민E"
Topic 3: 0.043*"제품용량:1개월분" + 0.039*"관절/뼈건강" + 0.038*"비타민D" + 0.028*"아연" + 0.018*"섭취량:1포"
Topic 4: 0.105*"캡슐" + 0.075*"혈행개선" + 0.055*"제품용량:1개월분" + 0.049*"비타민E" + 0.045*"섭취량:1캡슐"
Topic 5: 0.035*"비타민D" + 0.030*"비타민E" + 0.026*"아연" + 0.023*"비타민B6" + 0.023*"씹어서"
Topic 6: 0.040*"분말" + 0.038*"제품용량:1개월분" + 0.038*"비타민C" + 0.035*"섭취량:1포" + 0.031*"음용"
Topic 7: 0.051*"섭취량:1포" + 0.050*"음용" + 0.047*"제품용량:1개월분" + 0.035*"건강기능식품" + 0.025*"고형분"
Topic 8: 0.060*"비타민C" + 0.042*"아연" + 0.033*"비타민D" + 0.032*"비타민B2" + 0.031*"비타민B1"


--- 10 Topics ---
Topic 0: 0.047*"아연" + 0.031*"비타민D" + 0.025*"제품용량:1개월분" + 0.023*"비타민B2" + 0.022*"비타민B1"
Topic 1: 0.063*"음용" + 0.056*"섭취량:1포" + 0.034*"일반식품" + 0.034*"제품용량:1개월분" + 0.030*"건강기능식품"
Topic 2: 0.033*"섭취량:1포" + 0.029*"일반식품" + 0.028*"제품용량:15일분" + 0.027*"씹어서" + 0.024*"구미/젤리"
Topic 3: 0.074*"비타민C" + 0.046*"아연" + 0.034*"비타민E" + 0.031*"비타민B2" + 0.030*"비타민A"
Topic 4: 0.069*"캡슐" + 0.065*"혈행개선" + 0.042*"동물성" + 0.042*"비타민E" + 0.039*"제품용량:1개월분"
Topic 5: 0.041*"비타민C" + 0.032*"건강기능식품" + 0.028*"음용" + 0.025*"홍삼농축액" + 0.022*"면역력"
Topic 6: 0.092*"단백질성분" + 0.046*"섞어서" + 0.038*"맛" + 0.037*"단백질종류" + 0.027*"WPC"
Topic 7: 0.043*"비타민C" + 0.036*"아연" + 0.031*"비타민D" + 0.030*"캡슐" + 0.028*"비타민A"
Topic 8: 0.039*"비타민D" + 0.037*"비타민C" + 0.035*"비타민B6" + 0.032*"아연" + 0.030*"비타민B1"
Topic 9: 0.027*"비타민B2" + 0.027*"판토텐산" + 0.026*"나이아신" + 0.025*"아연" + 0.023*"비타민B1"
------------------------1번 제품------------------------
Best Topics for Coherence: [1, 7, 6]
Best Topics for Perplexity: 1


## **소분류**

In [ ]:
label_start = 1

for i in range(1, 54):
    num_str = f"{i:02}"

    df = pd.read_csv(f'/content/data/sub/B002-C003-00{num_str}_data.csv')

    # p_1 = df['제품특성'].apply(lambda x: [word for word in nltk.word_tokenize(str(x)) if word not in ['(', ',', ')', "'", ':', '%', '&', '[', ']']])
    p_1 = df['제품특성'].apply(lambda x: [word.strip("'") for word in nltk.word_tokenize(str(x)) if word not in ["''",'``',"'",'[', ',', ']', ':', '%', '&']])
    # 불용어 처리
    stop_words = set([
    "유형", "총","물과","종류_A",":6개","사료","살균방식용량연령연령연령연령연령연령연령타켓연령","풍부한","모발용","손상","충전방식","충전시간:1시간","풍부한","거품","부드러운","발림성","원료","편리한","사용성","형태용량","과일향","베이비파우더향","기타향","제형","날수부가기능부가기능부가기능:5중날",":3팩","종류구성날수헤드종류부가기능부가기능부가기능부가기능","종류구성헤드종류","칫솔모타입","민트향","안전","허브향","형태기능기능용량","알레르기","웨이","/","수확시기","초코","소재","-free","칫솔종류","간식에","두뇌발달","최소연령포장형태단계알레르기","단계없음","","구성형태용량","용량:200ml","용량:175ml","풍부함","촉촉함","종류구성날수헤드종류부가기능","상쾌함","색상","종류구성날수헤드종류부가기능부가기능부가기능","칫솔헤드크기","전원방식","종류특징","유무","전원방식","분당","작동방식","색상","칫솔모형태", "칫솔모강도","종류형태구성","가스","향","형태구성등급용도용도용도용도용량","30cm","세탁기유형","구성형태","투명정도","용량손잡이유무투명정도:5L","손잡이유무","면역력","250ml","사용횟수재질","방식","향","오","드","촉촉함", "사용횟수재질재질재질","메인향","모든피부용", "96","세","20g","섭취량제품용량주요","주원료","함량:60","없음","단백질성분","맛","단백질종류","섭취량:2캡슐","섭취량:2정","제품용량:2개월분","섭취량:1정","제품용량:1개월분", "섭취량","'1일", "함께","주성분","사용연령","분유성분","용기형태","형태구성용량","자외선차단지수","제품종류","사용부위","사용시간","살균방식","몸무게","사이즈","형태용량재질재질타켓연령연령연령연령연령","헤어타입","성별","용기형태","피부타입","추가구성","사용부위","물에","사이즈","종류구성날수헤드종류부가기능부가기능부가기능부가기능부가기능","사용대상","제품용량","일반식품","두","급여방법","저자극","성인남녀","x","제품유형","한","부가기능","제품타입사이즈","유발성분",":10팩","제품형태",":1팩","대상","g","성분" ,"구성용량","사용횟수","본품","형태","주요재질","식약처인증","하루","섭취횟수","식약처고시","영양소","원료명","횟수", "사용", "번","방법", "사용방법", "바로", "정", "기능", "함께", "주요","일반", "타입",
    "종류", '중량',"원료명","섭취대상","살균방식용량연령연령연령연령연령연령연령타켓연령","칫솔종류칫솔모타입","헤어","안전기능", "종류구성날수헤드종류부가기능부가기능부가기능부가기능부가기능부가기능","종류구성날수헤드종류부가기능부가기능","다름","개인별","':1개", "섭취방법",'교체','작동', '균일', '시즌', '증정', "밸런스","패턴", "상태", "가능성", "개인", "개당", "자동", "상관없음",
    "권장", "구분", "복합", "개선",":2개", "전용", "1일", ":1개","1개","방문", "901","부가", "이력", "모든", "수량", "출력", "기타", "필수", "해당", "시스템", "수시로", "제품", "품목",
    "구성","세트", "건강기능식품","유향","플라스틱","사용대상","타입","세부제품특징","주요제품특징","주요제품특징","매장위치주요소재","주요소재","주요","사용", "적용", "기능","특징", "혜택","조건", "단계", "용도", "과정","타겟",
    "모드", "관리", "기능성","섭취량제품용량주요","+","알림","용량","살균방식용량소비전력","프리","충전","저장매체","인터페이스","*","발림","부드러움","향계열","상태표시등","흡수력","세정력","면도방식","기능기능기능기능기능기능기능기능기능기능","제품타입섭취방법섭취대상섭취횟수1일","고정방식","편의기능","형태사용횟수주성분","무첨가","보관방법","최소연령",'포장형태','유발성분알레르기','인증', "부가기능","세트구성","크기", "재질","제품타입", "편의", "표시", "중심", "모형", "공용", "개월", "세부", "인기", "상품", "효과", "무게", "디테일", "기본", "편리", "인용","정성",
    "전체", "규격","날수부가기능부가기능부가기능부가기능부가기능:5중날","종류기능기능특징","기능기능기능기능기능기능","바나나","피부타입주요제품특징주요제품특징","피부타입주요제품특징주요제품특징용기형태용량","형태고정방식","우유","섭취량제품용량섭취횟수섭취방법섭취대상제품유형주요","섭취량섭취횟수제품용량","조리시간","섭취량제품용량섭취횟수섭취방법섭취대상제품유형주요","섭취량제품용량제품타입섭취횟수섭취방법섭취대상주요","캔","용기재질모양","타켓연령연령연령연령","타켓연령연령연령연령연령","구성구성구성재질특징특징타켓연령연령연령연령","종류재질타켓연령연령연령연령연령","용기재질모양모양모양","향","향계열","주요소재","소매기장","로고","핏"])

    p_1 = p_1.apply(lambda x: [word for word in x if word.lower() not in stop_words])

    # 데이터 전처리 후, 딕셔너리와 코퍼스 생성
    dictionary = Dictionary(p_1)
    corpus = [dictionary.doc2bow(text) for text in p_1]

    # 토픽 수 범위 설정
    start_topic_count = 1
    end_topic_count = 10
    step = 1

    # 일관성 점수 및 퍼플렉시티 계산
    coherence_scores = []
    perplexity_scores = []
    scores = []

    for num_topics in range(start_topic_count, end_topic_count + 1, step):
        model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary)

        coherence_model = CoherenceModel(model=model, texts=p_1, dictionary=dictionary, coherence='c_v')
        coherence_score = coherence_model.get_coherence()
        coherence_scores.append((num_topics, coherence_score))

        perplexity_score = model.log_perplexity(corpus)
        perplexity_scores.append((num_topics, perplexity_score))

        scores.append((num_topics, coherence_score, perplexity_score))

        # 토픽 별 단어 출력
        print(f"--- {num_topics} Topics ---")
        for topic_id, words in model.print_topics(num_words=5):
            print(f"Topic {topic_id}: {words}")

    print(f'------------------------{i}번 제품------------------------')
    # 일관성 점수 및 퍼플렉시티 점수 출력
    for num_topics, (coherence_score, perplexity_score) in zip(range(start_topic_count, end_topic_count + 1, step), zip(coherence_scores, perplexity_scores)):
        print(f"Number of Topics: {num_topics}, Coherence Score: {coherence_score}, Perplexity Score: {perplexity_score}")

    # coherence 점수 상위 3개 중 perplexity가 가장 높은 토픽 선택
    top_scores = sorted(scores, key=lambda x: x[1], reverse=True)[:3]
    best_num_topics = max(top_scores, key=lambda x: x[2])[0]

    print(f"Best Topics for Coherence: {[score[0] for score in top_scores]}")
    print(f"Best Topics for Perplexity: {best_num_topics}")

    best_model = LdaModel(corpus, num_topics=best_num_topics, id2word=dictionary)

    # 각 문서에 대한 토픽 분포 가져오기
    doc_topic_dists = [best_model.get_document_topics(doc) for doc in corpus]

    # 가장 확률이 높은 토픽 선택하기
    labels = [max(dist, key=lambda x: x[1])[0] + label_start for dist in doc_topic_dists]

    # 데이터프레임에 'label' 열 추가하기
    df['label'] = labels

    label_start += best_num_topics

    # Save the labeled data to a CSV file
    output_path = f'/content/data/sub/B002-C003-00{num_str}_data.csv'
    df.to_csv(output_path, index=False)

--- 1 Topics ---
Topic 0: 0.031*"감소" + 0.031*"체지방" + 0.029*"700mg" + 0.029*"28정" + 0.024*"분말"


--- 2 Topics ---
Topic 0: 0.041*"감소" + 0.040*"체지방" + 0.037*"28정" + 0.029*"700mg" + 0.027*"비타민C"
Topic 1: 0.038*"분말" + 0.031*"어류" + 0.030*"음용" + 0.029*"700mg" + 0.027*"섭취량:1포"


--- 3 Topics ---
Topic 0: 0.034*"음용" + 0.034*"체지방" + 0.033*"감소" + 0.030*"분말" + 0.025*"액상"
Topic 1: 0.099*"700mg" + 0.097*"28정" + 0.025*"실온보관" + 0.024*"섞어서" + 0.017*"끓는물"
Topic 2: 0.038*"감소" + 0.038*"체지방" + 0.034*"비타민C" + 0.029*"어류" + 0.027*"분말"


--- 4 Topics ---
Topic 0: 0.044*"실온보관" + 0.038*"전자레인지" + 0.035*"음용" + 0.029*"끓는물" + 0.028*":2분"
Topic 1: 0.076*"700mg" + 0.076*"28정" + 0.026*"체지방" + 0.025*"감소" + 0.024*"비타민C"
Topic 2: 0.047*"감소" + 0.047*"체지방" + 0.040*"어류" + 0.034*"섭취량:1포" + 0.032*"비타민C"
Topic 3: 0.045*"체지방" + 0.044*"감소" + 0.036*"분말" + 0.025*"섭취량:1포" + 0.021*"섞어서"


--- 5 Topics ---
Topic 0: 0.028*"비타민C" + 0.027*"감소" + 0.026*"체지방" + 0.025*"음용" + 0.025*"액상"
Topic 1: 0.068*"체지방" + 0.067*"감소" + 0.060*"28정" + 0.060*"700mg" + 0.031*"판토텐산"
Topic 2: 0.062*"분말" + 0.048*"섭취량:1포" + 0.030*"30포" + 0.028*"식이섬유" + 0.026*"어류"
Topic 3: 0.050*"실온보관" + 0.043*"전자레인지" + 0.034*"끓는물" + 0.032*":2분" + 0.028*":10분"
Topic 4: 0.084*"28정" + 0.084*"700mg" + 0.047*"비타민C" + 0.036*"분말" + 0.036*"어류"


...

--- 6 Topics ---
Topic 0: 0.195*"실온보관" + 0.082*"대두" + 0.081*"파우치" + 0.064*"쇠고기" + 0.059*"밀"
Topic 1: 0.198*"냉장보관" + 0.073*"대두" + 0.062*"밀" + 0.052*"아기국" + 0.050*"최소연령:10개월"
Topic 2: 0.219*"냉장보관" + 0.166*"200g" + 0.165*"최소연령:12개월" + 0.157*"진밥" + 0.037*"아기국"
Topic 3: 0.135*"실온보관" + 0.122*"파우치" + 0.084*"100g" + 0.073*"유발성분없음" + 0.061*"죽"
Topic 4: 0.153*"실온보관" + 0.080*"파우치" + 0.074*"유발성분없음" + 0.047*"쇠고기" + 0.040*"쌀가루"
Topic 5: 0.091*"쇠고기" + 0.080*"대두" + 0.076*"파우치" + 0.073*"밀" + 0.072*"실온보관"


--- 7 Topics ---
Topic 0: 0.163*"냉장보관" + 0.076*"실온보관" + 0.070*"진밥" + 0.057*"최소연령:12개월" + 0.046*"200g"
Topic 1: 0.107*"대두" + 0.101*"밀" + 0.098*"쇠고기" + 0.088*"실온보관" + 0.065*"파우치"
Topic 2: 0.099*"냉장보관" + 0.057*"150g" + 0.043*"대두" + 0.043*"쇠고기" + 0.043*"밀"
Topic 3: 0.069*"유발성분보관방법:4개월" + 0.054*"대두" + 0.053*"유발성분없음" + 0.046*"냉장보관" + 0.040*"실온보관"
Topic 4: 0.138*"유발성분없음" + 0.135*"실온보관" + 0.112*"퓨레" + 0.070*"파우치" + 0.054*"100g"
Topic 5: 0.098*"냉장보관" + 0.096*"죽" + 0.075*"쇠고기" + 0.037*"최소연령:7개월" + 0.033*"실온보관"
Topic 6: 0.219*"냉장보관" + 0.206*"진밥" + 0.201*"200g" + 0.201*"최소연령:12개월" + 0.010*"죽"


--- 8 Topics ---
Topic 0: 0.111*"냉장보관" + 0.094*"대두" + 0.078*"밀" + 0.051*"아기반찬" + 0.045*"쇠고기"
Topic 1: 0.252*"냉장보관" + 0.074*"진밥" + 0.064*"최소연령:12개월" + 0.063*"200g" + 0.052*"죽"
Topic 2: 0.239*"냉장보관" + 0.104*"진밥" + 0.081*"최소연령:12개월" + 0.080*"200g" + 0.061*"죽"
Topic 3: 0.103*"실온보관" + 0.084*"밀" + 0.079*"파우치" + 0.078*"대두" + 0.072*"쇠고기"
Topic 4: 0.105*"대두" + 0.099*"냉장보관" + 0.092*"밀" + 0.089*"실온보관" + 0.062*"아기국"
Topic 5: 0.137*"실온보관" + 0.093*"쇠고기" + 0.092*"파우치" + 0.055*"대두" + 0.044*"최소연령:12개월"
Topic 6: 0.097*"냉장보관" + 0.061*"아기밥" + 0.061*"최소연령:15개월" + 0.049*"실온보관" + 0.049*"150g"
Topic 7: 0.110*"실온보관" + 0.077*"파우치" + 0.062*"대두" + 0.061*"쇠고기" + 0.061*"최소연령:12개월"


--- 9 Topics ---
Topic 0: 0.115*"냉장보관" + 0.097*"진밥" + 0.095*"200g" + 0.087*"실온보관" + 0.078*"최소연령:12개월"
Topic 1: 0.133*"냉장보관" + 0.105*"200g" + 0.097*"최소연령:12개월" + 0.088*"진밥" + 0.045*"밀"
Topic 2: 0.159*"실온보관" + 0.090*"유발성분없음" + 0.062*"대두" + 0.062*"파우치" + 0.060*"밀"
Topic 3: 0.185*"냉장보관" + 0.103*"진밥" + 0.101*"최소연령:12개월" + 0.093*"200g" + 0.034*"대두"
Topic 4: 0.142*"실온보관" + 0.137*"파우치" + 0.071*"쇠고기" + 0.070*"대두" + 0.057*"최소연령:12개월"
Topic 5: 0.253*"냉장보관" + 0.080*"죽" + 0.060*"미음" + 0.054*"최소연령:13개월" + 0.036*"최소연령:7개월"
Topic 6: 0.119*"쇠고기" + 0.054*"대두" + 0.041*"최소연령:12개월" + 0.041*"진밥" + 0.041*"밀"
Topic 7: 0.115*"냉장보관" + 0.072*"유발성분보관방법:4개월" + 0.068*"대두" + 0.055*"밀" + 0.051*"유발성분없음"
Topic 8: 0.122*"냉장보관" + 0.103*"대두" + 0.090*"밀" + 0.080*"아기국" + 0.077*"쇠고기"


--- 10 Topics ---
Topic 0: 0.233*"냉장보관" + 0.066*"아기국" + 0.047*"밀" + 0.045*"최소연령:10개월" + 0.045*"대두"
Topic 1: 0.097*"파우치" + 0.081*"쇠고기" + 0.070*"대두" + 0.055*"밀" + 0.050*"실온보관"
Topic 2: 0.195*"진밥" + 0.176*"200g" + 0.167*"최소연령:12개월" + 0.167*"냉장보관" + 0.039*"파우치"
Topic 3: 0.141*"최소연령:12개월" + 0.120*"진밥" + 0.118*"200g" + 0.110*"냉장보관" + 0.074*"실온보관"
Topic 4: 0.123*"실온보관" + 0.109*"대두" + 0.082*"쇠고기" + 0.082*"밀" + 0.067*"파우치"
Topic 5: 0.142*"실온보관" + 0.097*"파우치" + 0.056*"유발성분없음" + 0.047*"100g" + 0.040*"쇠고기"
Topic 6: 0.081*"쇠고기" + 0.075*"냉장보관" + 0.060*"실온보관" + 0.056*"대두" + 0.046*"닭고기"
Topic 7: 0.089*"냉장보관" + 0.071*"최소연령:15개월" + 0.059*"실온보관" + 0.044*"대두" + 0.044*"밀"
Topic 8: 0.123*"냉장보관" + 0.112*"최소연령:12개월" + 0.107*"진밥" + 0.090*"200g" + 0.057*"파우치"
Topic 9: 0.116*"실온보관" + 0.071*"유발성분없음" + 0.071*"최소연령:6개월" + 0.048*"냉장보관" + 0.048*"죽"
------------------------53번 제품------------------------
Number of Topics: 1, Coherence Score: (1, 0.3427822338267342), Perplexity Score: (1, -3.794545615790239)
Number of T

In [ ]:
# LDA labeling
frames = [pd.read_csv(f'/content/data/sub/B002-C003-00{i:02}_data.csv') for i in range(1, 54)]

# 모든 데이터프레임을 하나로 합칩니다.
result = pd.concat(frames)

# 결과를 새 CSV 파일에 저장합니다.
result.to_csv('/content/data/LDA_data.csv', index=False)